# CSX46 Class Notebook 12 - co-analyzing the human PPI network and Gene Ontology "biological process" annotations

In this notebook exercise, we will (inspired by Figure 1C in Rhodes et al.) compute the likelihood ratio (LR) for human protein-protein pairs to have an interaction (vs. not have an interaction), given the size (i.e., number of genes) of the smallest (i.e., most specific) Gene Ontology biological process term that the two proteins share in common. To do this, we will be co-analyzing the human protein-protein interaction network from Pathway Commons and protein annotations (Gene Ontology biological process) from UniprotKB. Your analysis will produce empirical estimates of the LR for interacting/not-interacting, as a function of the size `size_smallest_shared_bp_int` of the smallest GO biological process shared by a given protein pair; note, as in the Rhodes et al. paper, this will be done in a discrete binning of the `size_smallest_shared_bp_int` value, given limitations on the data set size.


Import the python module dependencies for this notebook: pandas as pd, numpy as np, random, and pprint

In [1]:
import pandas as pd
import numpy as np
import random
import pprint

From S3, download the Pathway Commons database and uncompress it

In [2]:
!curl https://csx46.s3-us-west-2.amazonaws.com/PathwayCommons9.All.hgnc.sif.gz \
      --output PathwayCommons9.All.hgnc.sif.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5930k  100 5930k    0     0  12.2M      0 --:--:-- --:--:-- --:--:-- 12.2M


Using the exclamation point to run `gunzip` at the Linux shell, uncompress the file `PathwayCommons9.All.hgnc.sif.gz`, producing an uncompressed file `PathwayCommons9.All.hgnc.sif`.

In [3]:
!gunzip -f PathwayCommons9.All.hgnc.sif.gz

From S3, download the UniprotKB proteome database (a tab-separated file of protein information)

In [4]:
!curl https://csx46.s3.us-west-2.amazonaws.com/uniprotkb_proteome_UP000005640_2024_01_29.tsv \
      --output uniprotkb_proteome_UP000005640_2024_01_29.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.7M  100 24.7M    0     0  25.5M      0 --:--:-- --:--:-- --:--:-- 25.5M--:--  0:00:01 20.7M


Using Linux shell tools `head`, `sed`, and `nl`, take a peek at the first row of the file, to get an enumerated list of the column names. Don't forget to use an exclamation point so that your line of code is run in the Linux shell and not in the python interpreter.

In [5]:
!head -1 uniprotkb_proteome_UP000005640_2024_01_29.tsv | sed 's/\t/\n/g' | nl

     1	Entry
     2	Reviewed
     3	Entry Name
     4	Protein names
     5	Gene Names
     6	Organism
     7	Length
     8	Gene Ontology (cellular component)
     9	Gene Ontology (biological process)
    10	Gene Ontology (molecular function)


Load the Pathway Commons SIF file `PathwayCommons9.All.hgnc.sif` into a Pandas dataframe `sif_data`, renaming the three columns `species1`, `interaction_type`, and `species2`

In [6]:
sif_data = pd.read_csv("PathwayCommons9.All.hgnc.sif",
                       sep="\t", names=["species1",
                                        "interaction_type",
                                        "species2"])

process the protein-protein interaction data to eliminate duplicates

In [7]:
interaction_types_ppi = set(["interacts-with",
                             "in-complex-with"])
interac_ppi = sif_data[sif_data.interaction_type.isin(interaction_types_ppi)].copy()
inds_swap = interac_ppi['species1'] > interac_ppi['species2']
interac_ppi.loc[inds_swap, ['species1', 'species2']] = interac_ppi.loc[inds_swap, ['species2', 'species1']].values
interac_ppi_unique = interac_ppi[["species1", "species2"]].drop_duplicates()

load a tab-separated data file `uniprotkb_proteome_UP000005640_2024_01_29.tsv` of protein information into a dataframe `prot_data`, retaining only columns `Gene Names` and `Gene Ontology (biological process)`

In [8]:
prot_data = pd.read_csv("uniprotkb_proteome_UP000005640_2024_01_29.tsv",
                        sep="\t")[['Gene Names', 'Gene Ontology (biological process)']]

make a dictionary `gene_names_dict` whose key is a gene name, and whose value is a list of Gene Ontology biological process annotation terms for that gene (fill in the steps below as indicated in the code comments with `TODO`)

In [10]:
ctr = 0
gene_names_dict = dict()

# TODO: using a `for` loop,
# iterate over prot_data using the `iterrows` method,
# in each iteration assigning the row index to `index` and
# the tuple of values of the row of the dataframe, to `row`
for index, row in prot_data.iterrows():
    ctr += 1
    gene_names_str = row.iloc[0]
    if isinstance(gene_names_str, float) and np.isnan(gene_names_str):
        continue
    gene_names_list = []
    # split the string `gene_names` by instances of the space (` `)
    # character, and assign the first (i.e., 0th in python indexing)
    # substring to the varible name `gene`
    gene = gene_names_str.split(' ')[0]
    if '-' in gene:
        # sometimes, the canonical gene name is a pair of
        # official gene symbols with a hyphen ("-") between them;
        # need to handle this case using an inelegant hack:
        if all(len(gn) > 3 for gn in gene_name.split('-')):
            for gn in gene_name.split('-'):
                gene_names_list.append(gn)
        else:
            gene_names_list.append(gene)
    else:
        gene_names_list.append(gene)
    go_bp = row.iloc[1]
    if not isinstance(go_bp, float) or not np.isnan(go_bp):
        # TODO: split up the string `go_bp` by occurrences
        # of the separator character `";"`, and for each
        # resulting substring, strip whitespace using the
        # `string.strip()` instance method and add it to
        # a final list `go_bp_list`
        go_bp_list = [bp.strip() for bp in go_bp.split(';')]
    else:
        go_bp_list = []
    for gene_name in gene_names_list:
        gene_names_dict[gene_name] = go_bp_list

starting with `gene_names_dict`, make a data frame `go_df` containing just two columns, `gene` and `bp`,
where `gene` contains the gene symbol and `bp` contains a list
of GO biological process term annotations for that gene symbol

In [11]:
go_df = pd.DataFrame({'gene': gene_names_dict.keys(),
                      'bp': gene_names_dict.values()})
go_df.head(n=10)

,gene,bp
0,DMD,[]
1,DGKI,[]
2,CYP2D7,[]
3,PTGS1,[response to oxidative stress [GO:0006979]]
4,HNF1A,[]
5,PTPN22,[]
6,PIGBOS1,[]
7,SIK1B,[activation of protein kinase activity [GO:003...
8,CD300H,[neutrophil chemotaxis [GO:0030593]]
9,FASN,[]


using `pandas.Series.to_dict()`, `pandas.DataFrame.groupby`, `pandas.DataFrameGroupBy.apply`, and `np.sum`,
make a dictionary `gene_to_go` relating the canonical gene name to the list of GO biological
process term annotations for that gene

In [ ]:
gene_to_go = pd.Series.to_dict(go_df.groupby([go_df.gene]).bp.apply(np.sum))

make a dictionary `go_to_gene` relating GO biological process terms to genes that
are annotated with the GO biological process term in the key (this is known as  "inverting" a dictionary); so, the key in this dictionary will be a Gene Ontology (GO) term and the value will be a Python list of gene symbols; it may be convenient to use a double for loop (outer loop is over key/value pairs `g`, `tl` returned by `gene_to_go.items()`, and the inner loop is over list elements of the `tl` list)

In [ ]:
go_to_gene = dict()
for g, tl in gene_to_go.items():
    for t in tl:
        go_to_gene[t] = go_to_gene.get(t, []) + [g]

calculate, for all pairs of interacting proteins (mapped to gene names),
the size of the smallest shared GO biological process annotation for the genes (fill in the steps below as indicated in the code comments with `TODO`)

In [ ]:
size_smallest_shared_bp_int = []
no_shared_bp_int = 0
int_set = set()  # we will need a set of "keys" of interacting proteins; we will use it later
for row in interac_ppi_unique.iterrows():
    g1 = row[1].species1
    g2 = row[1].species2
    int_set.add(g1 + '-' + g2)
    go1 = set(gene_to_go.get(g1, []))
    go2 = set(gene_to_go.get(g2, []))
    go12_terms = go1 & go2
    if len(go12_terms) > 0:
        # TODO: use dictionary comprehension to process the set `go12_terms` to
        # create a dictionary `go12_terms_sizes` mapping GO biological process terms
        # `t` from the set, to values defined by `len(go_to_gene[t])`:
        go12_terms_sizes = {t: len(go_to_gene[t]) for t in go12_terms}
        # TODO: using the python builtin `min` with the optional function
        # argument `key`, find the key (which we'll assign to `min_term`)
        # in `go12_terms_sizes` whose associated value is the smallest of any
        # value in that dictionary; assign that value to `size_min_term`
        min_term = min(go12_terms_sizes, key=go12_terms_sizes.get)
        size_min_term = go12_terms_sizes[min_term]
        # TODO: append `size_min_term` to the list `size_smallest_shared_bp_int`
        size_smallest_shared_bp_int.append(size_min_term)
    else:
        no_shared_bp_int += 1

calculate, for ten million random pairs of *non-interacting* proteins (mapped to gene names),
the size of the smallest shared GO biological process annotation for the genes

In [ ]:
size_smallest_shared_bp_no_int = []
no_shared_bp_no_int = 0
all_genes = list(gene_to_go.keys())
ctr = 0
Nnoint = 10000000
while ctr < Nnoint:
    g1 = random.choice(all_genes)
    g2 = g1
    while g2 == g1 or (g1 + '-' + g2) in int_set:  # use the "key" to check if they are interacting
        g2 = random.choice(all_genes)
    go1 = set(gene_to_go.get(g1, []))
    go2 = set(gene_to_go.get(g2, []))
    go12_terms = go1 & go2
    if len(go12_terms) > 0:
        go12_terms_sizes = {t: len(go_to_gene[t]) for t in go12_terms}
        min_term = min(go12_terms_sizes, key=go12_terms_sizes.get)
        size_min_term = go12_terms_sizes[min_term]
        size_smallest_shared_bp_no_int.append(size_min_term)
    else:
        no_shared_bp_no_int += 1
    ctr += 1


use Numpy's histogram feature to calculate the likelihood ratios using
the same binning based on GO biological process gene-set size that
Rhodes et al. used (fill in the steps below as indicated in the code comments with `TODO`)

In [ ]:
breaks = [0, 5, 10, 50, 100, 500]
Nint = interac_ppi_unique.shape[0]
l_no_shared = (no_shared_bp_int / Nint)/(no_shared_bp_no_int / Nnoint)
hist_int = np.histogram(size_smallest_shared_bp_int, bins=breaks)
# TODO: using `np.histogram`, create a histogram `host_no_int` from
# `size_smallest_shared_bp_no_int` using the set of breaks `breaks`,
# similarly to how we created `hist_int`
hist_no_int = np.histogram(size_smallest_shared_bp_no_int, bins=breaks)
# TODO: define list `l_ratios` as the ratio `(hist_int[0])/Nint` over
# `(hist_no_int[0]/Nnoint)`
l_ratios = (hist_int[0]/Nint)/(hist_no_int[0]/Nnoint)
l_ratios_res = [('no relation', l_no_shared)]
for ctr in range(len(l_ratios)):
    l_ratios_res.append((f"{breaks[ctr]}-{breaks[ctr+1]}",
                         float(l_ratios[ctr])))
pprint.pprint(l_ratios_res)

[('no relation', 0.9747489397849697),
 ('0-5', 28.667874085916512),
 ('5-10', 20.219375370115667),
 ('10-50', 14.71064327153566),
 ('50-100', 10.902083367879468),
 ('100-500', 5.358456449442843)]
